In [36]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import os
import re
import sys
import pandas as pd
import numpy as np
import random
from functools import partial
from pathlib import Path
from pprint import pprint
import copy
import seaborn as sns

import torch
import yaml
from torch.utils.data import DataLoader
from captum.attr import LRP 
from tqdm.std import tqdm

# for the LRP
import torch.nn as nn
from captum.attr import InputXGradient, LRP, IntegratedGradients
from captum.attr._utils.lrp_rules import (
    Alpha1_Beta0_Rule,
    EpsilonRule,
    GammaRule,
    IdentityRule,
)

from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score)
from sklearn.preprocessing import RobustScaler
import vis_pkg
import helper_pkg
import json
import cartopy as cartopy
import cartopy.crs as ccrs

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cmocean
plt.rcParams["font.family"] = "Arial"
import matplotlib as mpl
mpl.rcParams['hatch.linewidth'] = 0.3
mpl.rcParams.update({'hatch.color': 'gray'})

# Terrible hack to make sure Jupyter notebooks (which use different PYTHONPATH
# for some reason!) actually sees src/ directory so we can import from there.
os.chdir("/Users/jgra0019/Documents/codes/ml4szeq/ml4szeq")
print(f"--- Current working directory: {os.getcwd()}")
if not any([re.search("src$", path) for path in sys.path]):
    sys.path.append(str(Path.cwd() / "src"))

import default
from dataset import DFDataset
from fit import Fit
from utils import (convert_hyperparam_config_to_values, get_config,
                   get_full_hyperparam_config, load_data)
from model import *
from predictor import predictions

# %% GETTING HYPER-PARAMETERS
config_override_file = get_config("PARAMETER_YAML_FILE", None)
hyperparam_config = get_full_hyperparam_config(config_override_file=config_override_file)
print(
    "--- Hyperparameters/metadata set as follows (may be altered later if using EXISTING wandb sweep):"
)
pprint(hyperparam_config)


--- Current working directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq
--- Reading from default.yml to build up initial hyperparameters dictionary...
--- Checking to see if we want to override default hyperparameters with new file...
--- Override file found! Updating with hyperparameters from default.yml...
--- Hyperparameters/metadata set as follows (may be altered later if using EXISTING wandb sweep):
{'entity': 'jcgraciosa',
 'method': 'grid',
 'parameters': {'activation_function': {'value': 'relu'},
                'batch_normalisation': {'value': False},
                'batch_size': {'value': 16},
                'cat_scaling': {'value': 1},
                'categorical_output': {'value': True},
                'dataset': {'value': 'lin_samp_50'},
                'dropout': {'value': 0.2},
                'embeddings': {'value': 0},
                'epochs': {'value': 120},
                'exclude_file': {'value': 'xyz.csv'},
                'hidden_layers': {'value': [5

## Mapping code
All data are used in training, portion is reserved for testing

In [37]:
''' settings to define '''

tr_all_region_N_list = [0, 1, 2, 3, 4]
mod_rank_list = [1, 1, 1, 1, 1] 
region_list = ["cam", "sam"]
#region_list = ["alu", "cam", "izu", "ker", "kur", "ryu", "sam", "sum"]
num_class = 3 # number of classes

sep_dist = 250

# other important config
tr_half_use = None
do_tr_all_region = True

do_stnd = True          # perform standardization of the relevance values or not
apply_thresh = False    # set to True if we apply threshold to heatmap values

num_model = 1
device = "cpu"

model_dir = f"/Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls{num_class}"
out_dir = Path(f"{model_dir}/maps-sep{sep_dist}-cls{num_class}")

### A. Preparations for the map and function declarations

In [38]:
''' 
preparations for the map of the predictions
run before the loop since these don't really change
'''
# colormaps
cmap = matplotlib.cm.get_cmap('Set3') # colormap used
if num_class == 2:
        color_list = [  matplotlib.colors.rgb2hex(cmap(6/12)),  
                        matplotlib.colors.rgb2hex(cmap(3/12)) ]
elif num_class == 3:
        color_list = [  matplotlib.colors.rgb2hex(cmap(6/12)), 
                        matplotlib.colors.rgb2hex(cmap(5/12)), # 5/12 
                        matplotlib.colors.rgb2hex(cmap(3/12)) ]

# earthquake ruptures to map
eq_file_dict = {"alu": ["alu04", "alu06", "alu07", "alu08", "alu05"],
                "kur": ["kur00", "kur13", "kur02", "kur20", "kur12", "kur17", 
                        "kur14", "kur01", "kur19", "kur24", "kur03"],
                "sam": ["sam29", "sam11", "sam18", "sam24", "sam15", "sam07", "sam10",
                        "sam22", "sam09", "sam08", "sam04", "sam03", "sam06", "sam12",
                        "sam17", "sam26", "sam02", "sam28", "sam05", "sam19"],
                "sum": ["sum05", "sum04", "sum08", "sum00", "sum02",
                        "sum03", "sum06"],
                "cam": ["cam03", "cam10", "cam07", "cam11", "cam01", "cam09",
                        "cam17", "cam12", "cam04", "cam00", "cam15"],
                "ker": ["ker02", "ker01", "ker00"],
                "ryu": ["ryu00", "ryu01", "ryu02"],
                "izu": []               
            }


/var/folders/0_/fbgfh0h563nf0y4g61d675gs665gb7/T/ipykernel_66027/751418225.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap('Set3') # colormap used


### Perform predictions and put in a dictionary

In [39]:
# perform predictions

random.seed(43) # set the random seed 
pred_dict = {}

for mod_rank, tr_all_region_N in zip(mod_rank_list, tr_all_region_N_list):
    
    hparam_file = f"{model_dir}/model{tr_all_region_N}/model{tr_all_region_N}-sep{sep_dist}-cls{num_class}-top10.json"
    hparam_file = Path(hparam_file)

    with open(hparam_file) as json_data_file:
        hparam = json.load(json_data_file)

    hparam_sset = hparam

    hparam_sset = hparam_sset[str(mod_rank)]

    epoch_use = hparam_sset["epoch_use"]
    folder_use = hparam_sset["folder"]
    
    mod_dir = Path(f"{model_dir}/model{tr_all_region_N}")

    print(f"Model directory: {mod_dir}")
    print(f"Output directory: {out_dir}")

    ''' MACHINE LEARNING STUFF HERE '''

    # set-up hyperparameters - override values in default.yml 
    hyperparam_config["parameters"]["exclude_file"]["value"] = "xyz.csv" 
    hyperparam_config["parameters"]["dropout"]["value"] = float(hparam_sset["dropout"])
    hyperparam_config["parameters"]["hidden_layers"]["value"] = hparam_sset['hidden_layers']
    hyperparam_config["parameters"]["batch_size"]["value"] = hparam_sset["batch_sz"]
    hyperparam_config["parameters"]["learning_rate"]["value"] = float(hparam_sset["lr"])

    params, _ = convert_hyperparam_config_to_values(hyperparam_config) # convert here to include whatever were overriden
    data_suffix = params.get("dataset", "16k")  # which dataset you want as input
    data_folder = default.ROOT_DATA_DIRECTORY / data_suffix
    use_cache = False

    preprocessor = load_data(
            data_folder=data_folder,
            exclude_file="xyz.csv",
            target=params["target"],
            cats=params["mw_cats"],
            rand_seed = None, # for sampling with replacement 
            kernel_size=params["kernel_size"],
            use_cache=use_cache,
            rd_exclude = False if do_tr_all_region else True,
            protect_great=params["protect_great"],
            tr_half_use = tr_half_use,
            sep_dist = sep_dist,
            tr_all_region = False if do_tr_all_region else None,
            tr_all_region_N = tr_all_region_N
        )

        # Define arguments to be passed into our testing loop function.
    full_pred_kwargs = dict(
        df=preprocessor.dataframe,
        inputs=preprocessor.inputs,
        hyperparam_config=hyperparam_config,
        model_name_add=None,
        use_wandb=False,
    )

    # create pred_ds 
    pred_ds = DFDataset(dataframe = preprocessor.dataframe, 
                        inputs=preprocessor.inputs, 
                        target=preprocessor.target, 
                        force_cats = 5)
    pred_dl = torch.utils.data.DataLoader(pred_ds, batch_size = 1, shuffle=False)
    test_df = copy.deepcopy(pred_dl.dataset.dataframe)

    out_df = {}
    out_df["S_AVE"] = test_df["S_AVE"]
    out_df["LON_AVE"] = test_df["LON_AVE"]
    out_df["LAT_AVE"] = test_df["LAT_AVE"]
    out_df["MR_ISC"] = test_df["MR_ISC"]
    out_df["MR_GCMT"] = test_df["MR_GCMT"]
    out_df["MW_CAT"] = test_df["MW_CAT"]
    out_df["REGION"] = test_df["REGION_NAME"]

    ################## PERFORM PREDICTION 
    model_fname = "epoch-" + str(epoch_use) + ".pt"
    idx = 0

    model_path = mod_dir/hparam_sset["folder"]
        
    pred_obj = Fit(fit_on_init = False, **full_pred_kwargs, force_cats = 0) # initialize lang pirmi
    pred_model = copy.deepcopy(pred_obj.model)

    #loop through all 
    pred_model.load_state_dict(torch.load(model_path))
    pred_model.to(device)

    pred_model.eval()
    preds = np.zeros([len(pred_dl.dataset.dataframe), num_class])
    class_preds = np.zeros([len(pred_dl.dataset.dataframe)])

    # Loop through test data
    with torch.no_grad():
        for i, ((x_cont, x_region), (cat_labels, cont_labels)) in enumerate(pred_dl): 
            x_cont, x_region, cat_labels, cont_labels = (
                x_cont.to(device),
                x_region.to(device),
                cat_labels.to(device),
                cont_labels.to(device),
            )

            # Get outputs and calculate loss
            cat = pred_model(x_cont)
            pred_vals = torch.sigmoid(cat)
            _, pred_vals2 = torch.max(torch.sigmoid(cat), 1)
            #print(cat)

            preds[i] = pred_vals
            class_preds[i] = pred_vals2

        # save to dataframe for evaluation
        for i in range(num_class):
            out_df["MDL_"+ str(idx) + "_CLS_" + str(i)] = preds[:, i]

    out_df["MDL_" + str(idx) + "_PRED"] = class_preds
    idx += 1

    out_df = pd.DataFrame(out_df)
    out_df["CLASS_PRED"] = class_preds

    pred_dict[tr_all_region_N] = out_df

    ########## prepare the results
    # num_model = 1

    # col_list =["MDL_" + str(x) + "_CLS_0" for x in range(num_model)]
    # cls0_df = out_df[ ["S_AVE"] + col_list]
    # cls0_df = cls0_df.assign(MEAN=cls0_df[col_list].mean(axis = 1))
    # cls0_df = cls0_df.assign(STD=cls0_df[col_list].std(axis = 1))

    # col_list = ["MDL_" + str(x) + "_CLS_1" for x in range(num_model)]
    # cls1_df = out_df[["S_AVE"] + col_list]
    # cls1_df = cls1_df.assign(MEAN=cls1_df[col_list].mean(axis = 1))
    # cls1_df = cls1_df.assign(STD=cls1_df[col_list].std(axis = 1))

    # # col_list = ["MDL_" + str(x) + "_CLS_2" for x in range(num_model)]
    # # cls2_df = out_df[["S_AVE"] + col_list]
    # # cls2_df = cls2_df.assign(MEAN=cls2_df[col_list].mean(axis = 1))
    # # cls2_df = cls2_df.assign(STD=cls2_df[col_list].std(axis = 1))

    # col_list = ["MDL_" + str(x) + "_PRED" for x in range(num_model)]
    # pred_df = out_df[["S_AVE"] + col_list]


Model directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/model0
Output directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/maps-sep250-cls3
Dropping any columns which have fewer than  90% (531.9) values
Dropping any rows which are missing *any* input variables
Dropped 33 columns, and 35 rows.
Executing power transformer ... 


/Users/jgra0019/mambaforge/envs/earthquakes/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (556). n_quantiles is set to n_samples.
  warnings.warn(


Model directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/model1
Output directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/maps-sep250-cls3
Dropping any columns which have fewer than  90% (531.9) values
Dropping any rows which are missing *any* input variables
Dropped 33 columns, and 35 rows.
Executing power transformer ... 


/Users/jgra0019/mambaforge/envs/earthquakes/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (556). n_quantiles is set to n_samples.
  warnings.warn(


Model directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/model2
Output directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/maps-sep250-cls3
Dropping any columns which have fewer than  90% (531.9) values
Dropping any rows which are missing *any* input variables
Dropped 33 columns, and 35 rows.
Executing power transformer ... 


/Users/jgra0019/mambaforge/envs/earthquakes/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (556). n_quantiles is set to n_samples.
  warnings.warn(


Model directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/model3
Output directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/maps-sep250-cls3
Dropping any columns which have fewer than  90% (531.9) values
Dropping any rows which are missing *any* input variables
Dropped 33 columns, and 35 rows.
Executing power transformer ... 


/Users/jgra0019/mambaforge/envs/earthquakes/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (556). n_quantiles is set to n_samples.
  warnings.warn(


Model directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/model4
Output directory: /Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/maps-sep250-cls3
Dropping any columns which have fewer than  90% (531.9) values
Dropping any rows which are missing *any* input variables
Dropped 33 columns, and 35 rows.
Executing power transformer ... 


/Users/jgra0019/mambaforge/envs/earthquakes/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (556). n_quantiles is set to n_samples.
  warnings.warn(


### B. Loop through all the models, perform predictions, and map

In [40]:
conf_dir = Path("/Users/jgra0019/Documents/codes/region-paths")
map_set_fname = "/Users/jgra0019/Documents/codes/globdat-paths/map_setting_conf.json"

with open(map_set_fname) as data_file:
    #print(map_set_fname)
    all_map_setting = json.load(data_file)

In [41]:
dpi = 300

for region in region_list:

    # get all the data for current region
    region_df = []

    for tr_all_region_N in tr_all_region_N_list:

        df = pred_dict[tr_all_region_N]
        sset = df[df["REGION"] == region]
        region_df.append(sset)

    region_df = pd.concat(region_df).reset_index(drop = True)

    # config file
    conf_fname = conf_dir/(region + "_conf.json")
    with open(conf_fname) as json_data_file:
        conf = json.load(json_data_file)

    map_setting = all_map_setting[region]
    root_dir = Path(conf["mac_root"])
    other_settings = conf["conf"]
    conf = conf["path"]

    ##########################
    # Map settings
    ##########################

    # convert range of longitude to -180 to 180 if alu
    is_alu = False
    if any(re.findall(r'alu', region, re.IGNORECASE)):
        is_alu = True
    is_ker = False
    if any(re.findall(r'ker', region, re.IGNORECASE)):
        is_ker = True

    # read trench data
    trench_plt = pd.read_csv(root_dir/conf["trench_full"], sep=',', header = None)
    trench_plt.columns = ['LON', 'LAT']

    # text labels 
    label_fname = root_dir/conf["map_label"]
    label_df = pd.read_csv(label_fname, header = 'infer')
    label_df = label_df[label_df["TEXT"] != "Reference (0 km)"] # remove the reference - replace with a red marker
    label_df = label_df[label_df["IS_EQ"] != True] # remove earthquake labels
    label_df.loc[label_df["COLOR"] == 'yellow', "COLOR"] = 'firebrick'

    # earthquake slip files
    fdir = root_dir/conf["slip_line2"]

    eq_files = eq_file_dict[region]

    eq_outlines = {}
    # iterate opening
    cnt = 0
    for subdir, dirs, files in os.walk(fdir):
        for file in files:
            file_id = file.replace("_sqk_", "").replace("_rup.csv", "")
            if file_id in eq_files:
                fname = fdir/file
                eq_df = pd.read_csv(fname, sep = ',', comment = "#")
                #print(eq_df)
                #print(fname)
                if eq_df.shape[1] > 2:
                    eq_df = eq_df[['LON', 'LAT', 'COUNT']]
                    eq_df = eq_df.sort_values(by = 'COUNT')

                eq_outlines[cnt] = eq_df
                cnt += 1

    # other settings
    arr_sz = map_setting["trench_arr_sz"] # trench arrow size
    rel_pos = map_setting["label_rel_pos"] # for the label
    contour_fsize = map_setting["contour_fnt_sz"]  # contour label font size
    lwidth = map_setting["contour_lwidth"] # contour line width
    label_x = map_setting["region_lab_x"] # region label x
    label_y = map_setting["region_lab_y"] # region label y
    as_line = map_setting["eq_as_line"]

    # generate grid data since you need to do some revisions
    ''' Make sure these are correct!!! Especially ds value. '''
    n_max = 300 # in km in direction of downgoing plate
    n_min = -300 # in km in direction of upper plate
    dn = 300 # step in the n-axis
    ds = 50 # step in the s-axis 

    # Open the topo grid file
    # process topo grid file
    topo_fname = root_dir/conf['topo_grd']
    lon_topo, lat_topo, elev, topo_hs = vis_pkg.process_topo(topo_fname, set_lon360 = True if is_alu or is_ker else False)

    # since i don't want to write new code, make a work-around
    feat_df = pd.read_csv(root_dir/conf["dilat"], header =None, sep = "\t")
    feat_df = feat_df.dropna()
    feat_df = feat_df.reset_index(drop=True)
    feat_df.columns = ["LON", "LAT", "VAL"]
    feat_df["LON"] = feat_df["LON"]%360 # make sure to convert 


    n_ax, s_ax, lon_grid, lat_grid = helper_pkg.make_grid(root_dir/conf["grid_dir_" + str(ds)], 
                                                        n_max = n_max, n_min = n_min, 
                                                        dn = dn, ds = ds, 
                                                        ncdf_fname = None)
    lon_grid = lon_grid%360

    workaround_df = helper_pkg.map_data_to_grid(s_ax = s_ax, n_ax = n_ax, 
                                        lon_grid = lon_grid, lat_grid = lat_grid, 
                                        dep_df = None,  
                                        in_data_df = feat_df, 
                                        mode = 0, 
                                        rm_unmapped = True)

    # setting for the sort_by - for the PatchCollection
    if region in ["alu"]:
        sort_by = 0
    elif region in ["sam", "kur", "sum"]:
        sort_by = 1
    else:
        sort_by = 1

    ######### Create the map
    # just run every iteration since out_df is here - actually just need to run during first iteration
    workaround_df = workaround_df[workaround_df["S_AVE"] <= region_df["S_AVE"].max()]  
    #lon_grid = lon_grid%360 # convert to 0 - 360

    for s in region_df["S_AVE"].unique():
        val_neg = workaround_df[(workaround_df["S_AVE"] == s) & (workaround_df["N_AVE"] < 0)] 
        val_pos = workaround_df[(workaround_df["S_AVE"] == s) & (workaround_df["N_AVE"] > 0)] 
        
        region_df.loc[region_df["S_AVE"] == s, "LON_NEG"] = val_neg["LON_AVE"].max()
        region_df.loc[region_df["S_AVE"] == s, "LAT_NEG"] = val_neg["LAT_AVE"].max()

        region_df.loc[region_df["S_AVE"] == s, "LON_POS"] = val_pos["LON_AVE"].max()
        region_df.loc[region_df["S_AVE"] == s, "LAT_POS"] = val_pos["LAT_AVE"].max()

    region_df = region_df.sort_values(by = "S_AVE", ascending = True)

    ########## Assign colors
    for category in region_df["MW_CAT"].unique(): # expected class
        region_df.loc[region_df["MW_CAT"] == category, "EX_CLR"] = color_list[category]

    for category in region_df["CLASS_PRED"].unique(): # predicted class
        region_df.loc[region_df["CLASS_PRED"] == category, "ML_CLR"] = color_list[int(category)]
    
    ''' create the maps containing the predictions '''
    mapper = vis_pkg.Mapper(dpi = dpi,
                            data = None,
                            topo = {'LON': lon_topo, 'LAT': lat_topo, 'VAL': topo_hs},
                            extent = other_settings["map_extent"],
                            dim_inch = other_settings["map_wh_inch"],
                            is_alu = is_alu,
                            is_ker = is_ker
                            )
    mapper.create_basemap()
    mapper.add_trench_line(trench_df = trench_plt, linewidth = 0.7)
    mapper.add_trench_marker(trench_fname = root_dir/conf["trench_used"],
                            trench_arr_sz = arr_sz
                            )
    mapper.add_topo()
    # plot for the predicted class
    mapper.add_ml_1d_data_poly( df = region_df, 
                                lon_col = "LON_POS", lat_col = "LAT_POS", 
                                clr_col = "ML_CLR", sort_by = sort_by, alpha = 1)
    # plot for the expected class
    mapper.add_ml_1d_data_poly( df = region_df, 
                                lon_col = "LON_NEG", lat_col = "LAT_NEG", 
                                clr_col = "EX_CLR", sort_by = sort_by, alpha = 1, 
                                poly_edge_width = 0.1)
    mapper.add_slip_outlines(eq_outlines_dict = {'OUTLINE': eq_outlines, 
                                                'COLOR': "mediumblue", 
                                                'AS_LINE':True})
    mapper.add_map_labels(label_dict = {'LAB': label_df, 'REL_POS': rel_pos})

    # the legend
    leg_lab = [ r"$M_w < 8.0$", 
                r"$M_w \geq 8.0$"]

    patch_list = [mpatches.Patch(color = x, ec = "k", linewidth = 0.2, label = "C" + str(i) + ": " + lab) for i, (lab, x) in enumerate(zip(leg_lab, color_list))]

    if region in ["kur"]:
        legend = mapper.ax.legend(handles=patch_list, fontsize = 7, loc = "upper left")
        legend.get_frame().set_linewidth(0.2)

    # save the files
    out_dir.mkdir(parents=True, exist_ok=True) 
    map_fname = out_dir/(region + "-sep" + str(sep_dist) + ".png")
    print(map_fname)
    plt.savefig(map_fname, dpi = "figure", bbox_inches='tight')
    plt.close()


Reading  /Users/jgra0019/Documents/phd_research/data/topography/cam_region.grd
['Conventions', 'history', 'GMT_version', 'node_offset']
['lon', 'lat']
['lon', 'lat', 'z']


/Users/jgra0019/mambaforge/envs/earthquakes/lib/python3.11/site-packages/cartopy/mpl/gridliner.py:451: UserWarning: The .xlabels_top attribute is deprecated. Please use .top_labels to toggle visibility instead.
  warnings.warn('The .xlabels_top attribute is deprecated. Please '
/Users/jgra0019/mambaforge/envs/earthquakes/lib/python3.11/site-packages/cartopy/mpl/gridliner.py:487: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


/Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/maps-sep250-cls3/cam-sep250.png
Reading  /Users/jgra0019/Documents/phd_research/data/topography/sam_region.grd
['Conventions', 'history', 'GMT_version', 'node_offset']
['lon', 'lat']
['lon', 'lat', 'z']


/Users/jgra0019/mambaforge/envs/earthquakes/lib/python3.11/site-packages/cartopy/mpl/gridliner.py:451: UserWarning: The .xlabels_top attribute is deprecated. Please use .top_labels to toggle visibility instead.
  warnings.warn('The .xlabels_top attribute is deprecated. Please '
/Users/jgra0019/mambaforge/envs/earthquakes/lib/python3.11/site-packages/cartopy/mpl/gridliner.py:487: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


/Users/jgra0019/Documents/codes/ml4szeq/ml4szeq/out/models/all-used-training-wbal-cls3/maps-sep250-cls3/sam-sep250.png


In [50]:
other_settings["map_extent"]

[-91, -56, -47, 11.5]

In [51]:
workaround_df["LON_AVE"]

24467     280.746725
24468     280.746725
24741     280.746725
24742     280.746725
24743     280.746725
             ...    
164534    285.973771
164535    285.973771
164536    285.973771
164537    285.973771
164538    285.973771
Name: LON_AVE, Length: 22875, dtype: float64

In [52]:
lon_topo

array([-90.975, -90.925, -90.875, -90.825, -90.775, -90.725, -90.675,
       -90.625, -90.575, -90.525, -90.475, -90.425, -90.375, -90.325,
       -90.275, -90.225, -90.175, -90.125, -90.075, -90.025, -89.975,
       -89.925, -89.875, -89.825, -89.775, -89.725, -89.675, -89.625,
       -89.575, -89.525, -89.475, -89.425, -89.375, -89.325, -89.275,
       -89.225, -89.175, -89.125, -89.075, -89.025, -88.975, -88.925,
       -88.875, -88.825, -88.775, -88.725, -88.675, -88.625, -88.575,
       -88.525, -88.475, -88.425, -88.375, -88.325, -88.275, -88.225,
       -88.175, -88.125, -88.075, -88.025, -87.975, -87.925, -87.875,
       -87.825, -87.775, -87.725, -87.675, -87.625, -87.575, -87.525,
       -87.475, -87.425, -87.375, -87.325, -87.275, -87.225, -87.175,
       -87.125, -87.075, -87.025, -86.975, -86.925, -86.875, -86.825,
       -86.775, -86.725, -86.675, -86.625, -86.575, -86.525, -86.475,
       -86.425, -86.375, -86.325, -86.275, -86.225, -86.175, -86.125,
       -86.075, -86.